In [1]:
import matplotlib.image
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib
from PIL import Image
import os
import time
import fnmatch
import moviepy.editor
from os.path import isfile, join

In [2]:
def capture_frames(videopath,duration = 10):

    cap = cv2.VideoCapture(videopath)
    i = 0
    initial_time = round(time.time())

    while(cap.isOpened()):
        ret, frame = cap.read()
        #prevents infinite looping incase video ends
        time_at_execution = round(time.time())
        if(time_at_execution - initial_time >= duration):
            break

        cv2.imwrite(f'D:/GAN4CVD/Generated_frames/frame_{i}.jpg', frame)
        i += 1

    cap.release()
    cv2.destroyAllWindows()
    

In [3]:
def generate_recolored_frames(path):
  
    images = os.listdir(path)
    model = load_model('model_2600d.h5')
    
    count = len(fnmatch.filter(os.listdir(path), '*.*'))

    for i in range(0, count):
        imagePath = f'frame_{i}.jpg'
        image = cv2.imread(path + '/' + imagePath)
        
        imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(imageRGB)

        image = image.resize((256, 256))
        image = np.array(image)
        image = np.reshape(image, (-1, 256, 256, 3))

        image = (image - 127.5)/127.5
        recolor = model.predict(image)
        recolor = (recolor+1)/2.0
        recolor = np.reshape(recolor, (256, 256, 3))

        matplotlib.image.imsave(f'D:/GAN4CVD/Recolored_frames/frame{i}.jpg', recolor)


In [4]:
def convert_frames_to_video(pathIn,pathOut,fps):
    
    frame_array = []
    count = len(fnmatch.filter(os.listdir(pathIn), '*.*'))

    for i in range(0,count):
        filename=pathIn +'/'+ f'frame{i}.jpg'    
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        out.write(frame_array[i])
        
    out.release()


In [5]:
def recolor(video_path, video_output,duration):
    
    capture_frames(video_path, duration)
    generate_recolored_frames('D:/GAN4CVD/Generated_frames')
    convert_frames_to_video('D:/GAN4CVD/Recolored_frames', video_output, 29)

    # Delete all frames so that they don't get mixed when the model is ran again
    dir = 'D:/GAN4CVD/Generated_frames'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

    dir = 'D:/GAN4CVD/Recolored_frames'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
     
    audio_editing(duration, video_path, video_output)
    

In [6]:
def audio_editing(duration, video,output):
    
    video=moviepy.editor.VideoFileClip(video)
    audio=video.audio
    audio.write_audiofile('audio.mp3')
    
    audio0=moviepy.editor.AudioFileClip('audio.mp3')
    audio1=audio0.cutout(duration,audio0.duration)
    audio1.write_audiofile('audio_clip1.mp3')
    
    video=moviepy.editor.VideoFileClip(output)
    video=video.subclip(0,duration)
    audio=moviepy.editor.AudioFileClip('audio_clip1.mp3').subclip(0,duration)
    final_clip=video.set_audio(audio)

    final_clip.write_videofile('final_output.mp4')
    

In [8]:
recolor('input.mp4','output.mp4',5)


MoviePy - Writing audio in audio.mp3


MoviePy - Done.
MoviePy - Writing audio in audio_clip1.mp3


MoviePy - Done.
Moviepy - Building video final_output.mp4.
MoviePy - Writing audio in final_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video final_output.mp4



Moviepy - Done !
Moviepy - video ready final_output.mp4
